# **Подготовка данных**

Для начала нам необходимо загрузить файлы в виде pandas DataFrame

In [1]:
# Загрузка локальных файлов

from google.colab import files
import io

uploaded = files.upload()

Saving musae_git_edges.csv to musae_git_edges (8).csv
Saving musae_git_target.csv to musae_git_target (8).csv


In [3]:
# Импорт модуля pandas
import pandas as pd

# Чтение csv-файлов
target = pd.read_csv(io.BytesIO(uploaded['musae_git_target.csv']))
edges = pd.read_csv(io.BytesIO(uploaded['musae_git_edges.csv']))

## **Работа с узлами** *(musae_git_target.csv)*

In [4]:
target.head()

,id,name,ml_target
0,0,Eiryyy,0
1,1,shawflying,0
2,2,JpMCarrilho,1
3,3,SuhwanCha,0
4,4,sunilangadi2,1


## **Работа со связями** *(musae_git_edges.csv)*

In [5]:
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,6067,1972
3,3,495
4,3,3358


Исходя из описания данных, связи представляют собой подписки пользователей друг на друга, то есть - ребра неориентированного графа. Так как в дальнейшем эти данные будут использоваться для построения ориентированного графа, необходимо добавить несколько строк, обратных имеющимся, чтобы отобразить взаимную подписку пользователей друг на друга. Возьмем 25% строк случайным образом и поменяем названия колонок местами.

In [27]:
reverse_edges = edges.sample(frac=0.25, random_state=1).rename(columns={"id_1": "id_2", "id_2": "id_1"})
reverse_edges.tail()

,id_2,id_1
16878,5089,2946
8000,2197,584
5048,1418,3038
2375,735,1558
17443,5366,2964


Добавим полученные строки в конец DataFrame

In [28]:
edges = pd.concat([edges, reverse_edges], ignore_index=True)
edges.head()

,id_1,id_2
0,1,237
1,1,2383
2,6067,1972
3,3,495
4,3,3358


In [29]:
edges.tail()

,id_1,id_2
36176,2946,5089
36177,584,2197
36178,3038,1418
36179,1558,735
36180,2964,5366


Начало таблицы осталось прежним, а окончание совпадает с последними строками reverse_edges.

Далее необходимо создать матрицу смежности в виде DataFrame, в котором строки содержат id пользователя-подписчика, id пользователя, на которого подписались, и значение типа bool в зависимости от того, существует ли такая подписка или нет. Для этого добавим в DataFrame существующих связей колонку edge, заполнив ее единицами.

In [30]:
edges["edge"] = 1
edges

,id_1,id_2,edge
0,1,237,1
1,1,2383,1
2,6067,1972,1
3,3,495,1
4,3,3358,1
...,...,...,...
36176,2946,5089,1
36177,584,2197,1
36178,3038,1418,1
36179,1558,735,1


Создадим матрицу смежности в виде DataFrame из всех возможных пар пользователей.

In [31]:
adjacency = pd.DataFrame(target["id"])
adjacency = adjacency.merge(adjacency, how="cross")
adjacency

,id_x,id_y
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
100019996,10000,9996
100019997,10000,9997
100019998,10000,9998
100019999,10000,9999


И объединим новую таблицу с таблицей существующих связей, взяв от результата merge список всех пар id и связи.

In [32]:
adjacency = adjacency.merge(edges, how="left", left_on=["id_x", "id_y"], right_on=["id_1", "id_2"], copy=False)
adjacency = adjacency[['id_x', 'id_y', 'edge']]
adjacency

,id_x,id_y,edge
0,0,0,NaN
1,0,1,NaN
2,0,2,NaN
3,0,3,NaN
4,0,4,NaN
...,...,...,...
100020859,10000,9996,NaN
100020860,10000,9997,NaN
100020861,10000,9998,NaN
100020862,10000,9999,NaN


У пар, данных о которых не было в исходном файле связей, образовалось значение NaN. Его необходимо заменить на 0.

In [33]:
adjacency["edge"] = adjacency["edge"].fillna(0)
adjacency

,id_x,id_y,edge
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,0,3,0.0
4,0,4,0.0
...,...,...,...
100020859,10000,9996,0.0
100020860,10000,9997,0.0
100020861,10000,9998,0.0
100020862,10000,9999,0.0


Проверяем, что количество единиц в полученном DataFrame совпадает с числом существующих связей в edges.

In [38]:
adjacency["edge"].value_counts()[1] == edges.shape[0]

True